In [5]:
import tensorflow as tf
import os
import numpy as np
from copy import deepcopy
import sys

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K


In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

file_index = 1
noise_level = 0.5
clean_data_size = 50
seed = 10 * file_index
additional_data_size = 1000
learning_rate = 0.0003

In [7]:
# load data from CIFAR10
def load_data(clean_data_size):
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

    # transform labels to one-hot vectors
    y_train = tf.contrib.keras.utils.to_categorical(y_train, 10)
    y_test = tf.contrib.keras.utils.to_categorical(y_test, 10)

    clean_index = []
    for label in range(10):
        positive_index = list(np.where(y_train[:, label] == 1)[0])
        clean_index = np.append(clean_index, np.random.choice(positive_index, clean_data_size, replace=False)).astype(
            int)

    x_clean = x_train[clean_index]
    y_clean = y_train[clean_index]
    x_train = np.delete(x_train, clean_index, axis=0)
    y_train = np.delete(y_train, clean_index, axis=0)
    return x_train, y_train, x_test, y_test, x_clean, y_clean


def generate_noise_labels(y_train, noise_level):
    num_noise = int(noise_level * y_train.shape[0])
    noise_index = np.random.choice(y_train.shape[0], num_noise, replace=False)
    label_slice = np.argmax(y_train[noise_index], axis=1)
    new_label = np.random.randint(low=0, high=10, size=num_noise)
    while sum(label_slice == new_label) > 0:
        n = sum(label_slice == new_label)
        new_label[label_slice == new_label] = np.random.randint(low=0, high=10, size=n)
    y_train[noise_index] = tf.contrib.keras.utils.to_categorical(new_label, 10)
    return y_train

def create_model(architecture, num_classes, learning_rate=learning_rate, dropout=0.5):
    model = Sequential()
    for layer_index in range(len(architecture)):
        layer = architecture[layer_index]
        if len(layer) == 3:
            if layer_index == 0:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), input_shape=(32, 32, 3),
                                 kernel_initializer='glorot_normal', activation='relu', padding='same'))
            else:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), kernel_initializer='glorot_normal',
                                 activation='relu', padding='same'))
            if layer_index < 3:
                model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        elif len(layer) == 1:
            if len(architecture[layer_index - 1]) == 3:
                model.add(Flatten())
            model.add(Dense(layer[0], activation='relu', kernel_initializer='glorot_normal'))
        else:
            print('Invalid architecture /(ㄒoㄒ)/~~')
    model.add(Dropout(dropout))
    if num_classes > 2:
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    elif num_classes == 2:
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)
    return model

In [8]:
def run_benchmark():
    # create record files
    dirs = 'record/noise_' + str(noise_level) + '_iteration_10_cleansize_' + str(clean_data_size) + '_add_' + str(additional_data_size) + '_lr_' + str(learning_rate) + '/test' + str(file_index) + '/'
    if not os.path.exists(dirs):
        os.makedirs(dirs)

    file_benchmark = open(dirs + 'file_benchmark.txt', 'w+')

    np.random.seed(seed)
    tf.set_random_seed(seed)

    x_train, y_train, x_test, y_test, x_clean, y_clean = load_data(clean_data_size)
    y_train_orig = deepcopy(y_train)
    y_train = generate_noise_labels(y_train, noise_level)


    # train benchmark
    architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
    benchmark_model = create_model(architecture, num_classes=10)

    History_benchmark = benchmark_model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=100,
                                          shuffle=True)

    file_benchmark.write('training accuracy' + '\n')
    file_benchmark.write(str(History_benchmark.history['acc']) + '\n')
    file_benchmark.write('test accuracy' + '\n')
    file_benchmark.write(str(History_benchmark.history['val_acc']) + '\n')
    file_benchmark.close()

In [9]:
for file_index in range(5):
    run_benchmark()

Train on 49500 samples, validate on 10000 samples
Epoch 1/100
49500/49500 [==============================] - 67s 1ms/step - loss: 2.2365 - acc: 0.1795 - val_loss: 1.8761 - val_acc: 0.4029
Epoch 2/100
49500/49500 [==============================] - 68s 1ms/step - loss: 2.1695 - acc: 0.2416 - val_loss: 1.8032 - val_acc: 0.4614
Epoch 3/100
12096/49500 [======>.......................] - ETA: 48s - loss: 2.1369 - acc: 0.2617

KeyboardInterrupt: 